# Solving an Exactly-One-True problem

## Defining the problem
Suppose we want to solve the following problem:

$$\textbf{Among three conditions, there is only one truth.}$$

Let's use three binary variables $a$, $b$, and $c$ to denote the conditions. 

The truth table that satisfies this condition is:

\begin{array}{c c c c } \hline
a & b & c & \\ \hline
0 & 0 & 0 & \\ \hline
1 & 0 & 0 & \checkmark\\ \hline
0 & 1 & 0 & \checkmark\\ \hline
1 & 1 & 0 & \\ \hline
0 & 0 & 1 & \checkmark\\ \hline
1 & 0 & 1 & \\ \hline
0 & 1 & 1 & \\ \hline
1 & 1 & 1 & \\ \hline
\end{array}

Note that we assign 1 to True and 0 to False.


## Converting to a QUBO problem
The above truth table can also be written as an equation:

$$a + b + c = 1$$

which is equivalent to $(a+b+c-1)^2 = 0$.

If we define the objective function as 

$$E(a,b,c) = (a+b+c-1)^2 = 2ab + 2ac + 2bc - a - b -c +1,$$
solving $\min E(a,b,c)$ will give us the optimal solution. Here we make use of the binary nature of variables, that is $a^2 = a$.

Note: Removing the constant $1$ from the objective function doesn't change the solution, therefore we will  minimize the following energy function in the rest of this notebook:

$$E(a,b,c) = 2ab + 2ac + 2bc - a - b -c $$



Here in our QUBO:
* Each variable is a node weighted with its linear coefficient
* Each quadratic term is an edge with a strength between the nodes

Then, our QUBO can be represented by the following undirected graph:

<img src="images/one_true1.png"/>


## Putting the QUBO on Chimera

### Chimera structure
The Chimera architecture comprises sets of connected unit cells, each with four horizontal qubits connected to four vertical qubits via couplers. Unit cells are tiled vertically and horizontally with adjacent qubits connected, creating a lattice of sparsely connected qubits, shown as follows:
<img src="https://cloud.dwavesys.com/qubist/docs/sys_intro_getting_started/_images/chimera.png">

The notation C$N$ refers to a Chimera graph consisting of an $N\times N$ grid of unit cells. The D-Wave 2000Q QPU supports a C16 Chimera graph: its 2048 qubits are logically mapped into a $16\times 16$ matrix of unit cells of 8 qubits.

In a D-Wave QPU, the set of qubits and couplers that are available for computation is known as the working graph. The yield (working qubits) of a __working graph__ is typically less than the total number of qubits and couplers that are fabricated and physically present in the QPU.

More descriptions about Chimera graph can be found [here](https://cloud.dwavesys.com/qubist/docs/sys_intro_getting_started/chimera/).

### Minor-embedding

<div class="alert alert-warning" role="alert" style="margin: 10px"> 
Our question is: How do we put this QUBO on D-Wave quantum computer?
</div>

<img src="images/one_true4.png">

A technique that can achieve this is called _minor-embedding_. A minor-embedding of a graph $G$ in a quantum hardware graph $U$ is a subgraph of $U$ such that $G$ can be obtained from it by contracting edges [Choi2010](https://arxiv.org/pdf/1001.3116.pdf). A minor-embedding of graph requires creating virtual qubit out of multiple physical qubits using a concept called _chaining_.

### Embedding a three-sided graph $K_3$
If you look at the Chimera graph, there are no triangles, all of the cycles are of at least length 4. 

Using the concept of minor-embedding, we can make use of the cycle of length 4 to _construct_ a complete graph $K_3$. The procedure is illustrated in the following graph, where a thick blue line is used to denote that $2$ and $6$ should act as one logical qubit for variable $a$, which implies we need to make sure that chaining them up doesn't contribute to the total energy, i.e.,

$$w_{26}(q_2 - q_6)^2 = 0$$

Expanding and carefully manipulating the LHS yields $-w_{26}q_2q_6 + 0.5w_{26}q_2 + 0.5w_{26}q_6$


<img src='images/k3_on_chimera.png'>

Here are some embedding rules that we should keep in mind:

* A logical qubit can be mapped to $N$ physical qubits as long as the physical qubits form a connected set.  We call this a chain.

* For each physical coupler connecting two physical qubits in the same chain, include QUBO terms to cause the two physical qubits to align.

* When a logical qubit is mapped to an $N$-qubit chain, divide the bias (weight) for the logical qubit by $N$ and apply that bias to each qubit in the chain.

* Split the coupling strength between two logical qubits over all available physical couplers connecting the chains corresponding to the logical qubits.

Following the embedding rules, we have:

\begin{array}{ccccc} \hline
qubit & 2 & 6 & 3 & 7 \\ \hline
bias & -0.5+0.5w_{26} &  -0.5+0.5w_{26} & -1 & -1 \\\hline
var & a & a & b & c\\\hline
\end{array}

\begin{array}{ccccc} \hline
coupler & (2,6) & (2,7) & (3,6) & (3,7) \\ \hline
weights & -w_{26} & 2 & 2 & 2\\\hline
\end{array}


Here, $w_{26}$ is the chain strength. Choosing a value of $w_{26} = 3$ is sufficient in this example. 

<div class="alert alert-warning" role="alert" style="margin: 10px"> 
$\textbf{NOTE:}$ Choosing the proper chain strength normally takes a few tries.
</div>

Remember that the programmable coupling strength is $[-1, 1]$. We need to scale down $w_{26}$ by 3, which will automatically scale all the biases and weights by 3 as well. The QUBO table becomes

\begin{array}{ccccc} \hline
qubit & 2 & 6 & 3 & 7 \\ \hline
bias & 1/3 &  1/3 & -1/3 & -1/3 \\\hline
var & a & a & b & c\\\hline
\end{array}

\begin{array}{ccccc} \hline
coupler & (2,6) & (2,7) & (3,6) & (3,7) \\ \hline
weights & -1 & 2/3 & 2/3 & 2/3\\\hline
\end{array}

<div class="alert alert-warning" role="alert" style="margin: 10px"> 
$\textbf{NOTE:}$ In Ocean, the parameter `auto_scale=True` is enabled by default to handle the scaling automatically. If you want to take care of the problem scaling by yourself, you should set `auto_scale=False`. Under the setting of `auto_scale=False`, submitting a problem with bias values and/or coupler strength values that are out of the correct ranges will raise errors.
</div>


## Submitting problems to the quantum computer
Now we can submit our problem to the quantum computer. 

In [1]:
from __future__ import print_function
from dwave.system.samplers import DWaveSampler

# create sampler.   
sampler = DWaveSampler(solver={'qpu': True})   # Some accounts need to replace this line with the next:
# sampler = DWaveSampler(token = 'my_token', solver='solver_name')

# define the QUBO problem
w = 3.
Q = {(2, 2):-0.5+0.5*w, (6,6):-0.5+0.5*w, (3,3):-1., (7,7):-1., 
     (2,7):2., (2,6):-w, (3,6):2., (3,7):2.}
print('Q:{}'.format(Q))
result = sampler.sample_qubo(Q, num_reads=1000)

Q:{(2, 2): 1.0, (6, 6): 1.0, (3, 3): -1.0, (7, 7): -1.0, (2, 7): 2.0, (2, 6): -3.0, (3, 6): 2.0, (3, 7): 2.0}


In [2]:
# look at the statistics of the samples
print('a a b c')

R = iter(result)
E = iter(result.data())
for line in result:
    sample = next(R)
    data = next(E)
    energy = data.energy
    occurrences = data.num_occurrences
    print(sample[2], sample[6], sample[3], sample[7], occurrences)

a a b c
0 0 1 0 351
0 0 0 1 296
1 1 0 0 353


<div class="alert alert-success" role="alert" style="margin: 10px"> 
Exercise: Can you test the sample statistics with different chain strength?
</div>

# Embedding tools
## Ocean's embedding package `minorminer`
In this example, we are going to show how we can embed a graph using Ocean's `minorminer` package. As a reminder, a complete graph is a simple undirected graph in which every pair of distinct vertices is connected by a unique edge.

<img src='images/k1_k5.png'>

In [3]:
# import package from dwave-system
from minorminer import find_embedding

# define a K_5 graph
S_size = 5
S = {}
for i in range(S_size):
    for j in range(S_size):
        S[(i, j)] = 1

# Get the set of couplers from our sampler
A = sampler.edgelist

# Embed our problem onto our set of couplers
embeddings = find_embedding(S, A, verbose=2, random_seed=100)

initialized
embedding found.
max chain length 2; num max chains=3
reducing chain lengths


In [4]:
# inspect the embeddings
print(embeddings)
print(len(embeddings))

{0: [998, 992], 1: [999, 994], 2: [993], 3: [996], 4: [995, 997]}
5


These embeddings will look something like this:

In [5]:
from IPython.display import IFrame
IFrame("images/k5.pdf", width=600, height=300)

<div class="alert alert-success" role="alert" style="margin: 10px"> 

__Exercise__: Embed a complete $K8$ graph

</div>

## Ocean's embedding tool `EmbeddingComposite`
Using the `EmbeddingComposite` tool, we can combine solving and embedding in one step.

For example, we can embed and solve our QUBO that we established in the beginning of this notebook for the Exactly-One-Truth Problem.  As a reminder, this QUBO or energy function is given by : $$E(a,b,c) = 2ab + 2ac + 2bc - a - b -c.$$



In [ ]:
# Import the EmbeddingComposite tool
from dwave.system.composites import EmbeddingComposite

# Define our QUBO dictionary
Q = {('a','b'): 2, ('a','c'): 2, ('b','c'): 2, ('a','a'):-1, ('b','b'):-1, ('c','c'): -1}

# Set up our sampler
sampler = EmbeddingComposite(DWaveSampler())   # Some accounts need to replace this line with the next:
# sampler = EmbeddingComposite(DWaveSampler(token = 'my_token', solver='solver_name'))

# Run our QUBO through the EmbeddingComposite tool to the QPU and return our results
result = sampler.sample_qubo(Q, chain_strength=3, num_reads=1000)

# Print our results
print('a b c')
R = iter(result)
E = iter(result.data())
for line in result:
    sample = next(R)
    data = next(E)
    energy = data.energy
    occurrences = data.num_occurrences
    print(sample['a'], sample['b'], sample['c'], occurrences)

# Examples of Embeddings: 

<div class="alert alert-warning" role="alert" style="margin: 10px"> 
    $\textbf{NOTE:}$ Make sure you carefully choose the chain strength!
<div>

### Largest bipartite graph with chain_length = 4

In [ ]:
from IPython.display import IFrame
IFrame("images/largest_bipartite_chain4.pdf", width=600, height=300)

### Largest bipartite graph with any chain length

In [ ]:
from IPython.display import IFrame
IFrame("images/largest_bipartite.pdf", width=600, height=300)

### Largest complete graph with chain_length=4

In [ ]:
from IPython.display import IFrame
IFrame("images/largest_clique_chain4.pdf", width=600, height=300)

### Largest complete graph with any chain length

In [ ]:
from IPython.display import IFrame
IFrame("images/largest_clique.pdf", width=600, height=300)

In [ ]:
# shutdown the kernel
# %reset -f